<h1 style="text-align:center;"><b>12 - Minimum-Maximum Latency Partitioning using Union-Find and Binary Search</b></h1>

### **Problem description**:
        Given a network of g-nodes data centers and g-edges bidirectional connections, the ith connection connects data centers g-from[i] and g-to[i] with a latency of g-weight[i]. The max-latency of a network its the maximum latency of any connection. Divide this network into k or fewer networks by removing some of the connections such that the maximum latencies of all the regions are minimized. Find the minimum possible value of the maximum max-latency of the networks formed.

<div style="text-align: center;">
<img src="Images/12 - Minimum-Maximum Latency Partitioning using Union-Find and Binary Search 001.jpg" alt="Illustration of Example 1" />
</div>

Example:

        g-nodes = 3
        g-from = [1, 2, 3]
        g-to = [2, 3, 1]
        g-weight = [4, 5, 3]
        k = 2
        getMinMaxLatency(g_nodes, g_from, g_to, g_weight, k) = 3

_Explanation:_

        k = 2, so graph is split in two. max latency of the network is 5, so the graph is split to minimize latency and the max latency of 2 new networks is returned.

<div style="text-align: center;">
<img src="Images/12 - Minimum-Maximum Latency Partitioning using Union-Find and Binary Search 002.jpg" alt="Illustration of Example 1" />
</div>
Example:

        g-nodes = 2
        g-from = [1]
        g-to = [2]
        g-weight = [3]
        k = 1
        getMinMaxLatency(g_nodes, g_from, g_to, g_weight, k) = 0

_Explanation:_
        k = 1, so graph is not split. max latency is the max latency of the whole network.


### **Solution:**

In [5]:
def getMinMaxLatency(g_nodes, g_from, g_to, g_weight, k):

    from collections import defaultdict
    import heapq

    # Edge list with weights
    edges = []
    for i in range(len(g_from)):
        edges.append((g_weight[i], g_from[i], g_to[i]))
    # Sort edges by weight
    edges.sort()
    
    # Union-Find Data Structure
    class UnionFind:
        def __init__(self, size):
            self.root = list(range(size))
            self.rank = [1] * size

        def find(self, x):
            if self.root[x] != x:
                self.root[x] = self.find(self.root[x])
            return self.root[x]

        def union(self, x, y):
            rootX = self.find(x)
            rootY = self.find(y)

            if rootX != rootY:
                if self.rank[rootX] > self.rank[rootY]:
                    self.root[rootY] = rootX
                elif self.rank[rootX] < self.rank[rootY]:
                    self.root[rootX] = rootY
                else:
                    self.root[rootY] = rootX
                    self.rank[rootX] += 1

        def count_components(self):
            unique_roots = {self.find(x) for x in range(len(self.root))}
            return len(unique_roots)

    def canDivide(max_latency):
        uf = UnionFind(g_nodes)
        for weight, fr, to in edges:
            if weight > max_latency:
                break
            uf.union(fr - 1, to - 1)  # adjust for 0-based index
        # We need the number of unique components to be <= k
        return uf.count_components() <= k

    # Binary search for the minimum feasible maximum latency
    low, high = edges[0][0], edges[-1][0]
    best_latency = high

    while low <= high:
        mid = (low + high) // 2
        if canDivide(mid):
            best_latency = mid
            high = mid - 1
        else:
            low = mid + 1

    return best_latency